# NB #0:  Pool Exploration / Invariant 

This notebook provides a framework to plot a standardized pool and various transactions

For more information check out the [Balancer Simulations documentation](https://token-engineering-balancer.gitbook.io/balancer-simulations/).

### C1.1 A variable Invariant (50/50 pool)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
#set up genesis state
#define initial target spot price 
x_bal = 1000 # initial balance
y_bal = 1000 # initial balance
x_weight = 0.5 #define weight
y_weight = 0.5 #define weight
s_f = 0.5 #swap fee
inv = (x_bal**x_weight)*(y_bal**y_weight) #calculate invariant
x_vals = np.linspace(0, x_bal*4, num=50) # take x_bal and create 50 values array, nice curve only works with values larger than initial target spot price
#create dataframe with based on x_vals
i_t = pd.DataFrame(x_vals, columns=['token_X'])
i_t['invariant'] = inv #braucht man in plot nicht, nur zur kontrolle

#create values for plot, add Y_balances according to current invariant
i_t['token_Y'] = (i_t.invariant/(i_t.token_X**x_weight))**(1/y_weight)# calculate corresponding y_vals according to invariant
i_t['swap'] = 0
plot = i_t.loc[1:49,:] #leave out inf result 

plot.head()

In [ ]:
#new curve with swaps
#TODO alle formeln nochmal prüfen, alle variablen
#TODO array erstellen und loop installieren
#TODO refactor with all transaction types, concat only at the very last step


x_swaps = [200, 200, 200, 200, 200]  #define an array of swaps for plots
swap_no = 1
iswap_t = pd.DataFrame()
for i in x_swaps:
        y_out = y_bal*(1-(x_bal/(x_bal +(i*(1-s_f))))**(x_weight/y_weight))
        x_bal = x_bal + i #update balance token x
        y_bal = y_bal - y_out #update balance token y
        inv = (x_bal**x_weight)*(y_bal**y_weight) #update inv
        x_vals = np.linspace(0, x_bal*4, num=100)#create new values for curve, *4 required to maintain enough x_bal for the plot, otherwise x-axis values to small, curve too short
        iswap_t['token_X'] = x_vals
        iswap_t['invariant'] = inv
        iswap_t['token_Y'] = (iswap_t.invariant/(iswap_t.token_X**x_weight))**(1/y_weight)# calculate corresponding y_vals according to invariant
        iswap_t['swap'] = swap_no
        plot = plot.append(iswap_t, ignore_index=True) 
        swap_no += 1

In [ ]:
#plot.loc[45:255,:]
plot

In [ ]:
#TODO add annotations with Spotprice in every round

#plot curves with all 
fig = px.line(plot, x="token_X", y="token_Y", color='swap', color_discrete_sequence=px.colors.sequential.Turbo)
fig.update_xaxes(range=[0, 3000])
fig.update_yaxes(range=[0, 3000])
fig.update_layout(height=500, width=560, title_text='<b>A variable Invariant</b>')
#fig.add_annotation(x=1000, y=1000, text="Spot Price 0")
#fig.add_annotation(x=1000-50, y=900, text="Spot Price 1")
fig.show()